In [1]:
from konlpy.tag import Mecab 

mec = Mecab(dicpath=r"C:\mecab\mecab-ko-dic")

In [2]:
import pandas as pd
import os

df = pd.read_csv("../week3/inputs/df3_2_artsCleanEtc.csv")


In [8]:
mec.pos('10년차')

[('10', 'SN'), ('년', 'NNBC'), ('차', 'NNG')]

In [ ]:

#초기 구조화: 필수 -> 이후 case로 분기
series_mec = df['art_title'].map(lambda x : mec.pos(x, flatten=False, join=False))

df_mec = pd.DataFrame(columns=['art_id','token'])

for i in range(len(series_mec)):
    print(f'level {i}')
    for j in range(len(series_mec[i])):
        df_mec = df_mec.append({'art_id':i, 'token': series_mec[i][j]}, ignore_index=True)
        
# 저장주의!!!
df_mec.to_csv('df_mec_title_all_2.csv',encoding='utf-8-sig', index=False)


### -> outputs/week3/df_mec_title_all.csv 저장됨
<br>

## Compound 로직 
#### input: df_mec_title_all

---

#### case
1. 명사간 -> comp_NNG_NNG
2. VA + EC -> comp_VA_EC
3. VA + EF
4. SN + NR + NNBC
5. SN + NR
6. VCN + EC
7. NNG + XSN + JKB
8. NNG + JKB
9. NNG + XSN
10. MM + NNBC
11. MM + NNG
12. VV + (EP+EF)
13. SN + NNBC
14. VA + ETM
15. XSV + EC
16. SL + NNG
17. ('1', 'SN'), ('년', 'NNBC'), ('차', 'NNG'), 

In [6]:
import os
import pandas as pd

df_mec_title = pd.read_csv('../../outputs/week3/df0_1_mec_title_all.csv')

#저장용 path 셋팅
os.chdir(path='./outputs/')

df_mec_title.head()

,art_id,token
0,0,"[('erp', 'SL')]"
1,0,"[('프로그램', 'NNG')]"
2,0,"[('오류', 'NNG')]"
3,0,"[('장애', 'NNG'), ('조치', 'NNG')]"
4,0,"[('방법', 'NNG')]"


In [ ]:
# # (CASE1) 복합명사 처리x 기본 형태소 단위 그대로 넣은 경우 비교용

# df_res_basic = pd.DataFrame(columns=['art_id','token','type'])

# for i in range(len(df_mec_title)):
#     for j in range(len(df_mec_title.loc[i][1])):
#         df_res_basic = df_res_basic.append({'art_id':df_mec_title.loc[i][0], 'token':df_mec_title.loc[i][1][j][0], 'type':df_mec_title.loc[i][1][j][1]},ignore_index=True)
               
# print(df_res_basic)

# # df_res_basic.to_csv('df_res_basic.csv', encoding='utf-8-sig', index=False)

In [ ]:
# sample
df_mec_title_sample = df_mec_title[: 100]

# (CASE2) mecab 배열상 type 연속성 기반 복합화 : VA + EC (EX. '어떻,게' / '있','나요')

df_res_VA_EC = pd.DataFrame(columns=['art_id','token','type'])

df_mec_title_sample


#
# df_res_NNG_NNP.to_csv('df_res_NNG_NNP.csv', encoding='utf-8-sig', index=False)

# print(df_res_NNG_NNP)

In [4]:
# (CASE3) mecab 배열상 type 연속성 기반 복합명사화 : NNG,NNP-NNG,NNP 혼재 CASE 
    # 문제: 3개 연달아 등장하는 명사 해결법
    # 문제: 연

df_res_NNG_NNP = pd.DataFrame(columns=['art_id','token','type'])
# df_res_NNG_NNP.append({'art_id':, 'token','type'})

for i in range(len(df_mec)):
    if len(df_mec.loc[i][1]) <= 1 :
        df_res_NNG_NNP = df_res_NNG_NNP.append({'art_id':df_mec.loc[i][0],'token':df_mec.loc[i][1][0][0],'type':df_mec.loc[i][1][0][1]},ignore_index=True)
        continue
    for j in range(1,len(df_mec.loc[i][1])):
        if (df_mec.loc[i][1][j-1][1] == 'NNG' or df_mec.loc[i][1][j-1][1] == 'NNP') and (df_mec.loc[i][1][j][1] == 'NNG' or df_mec.loc[i][1][j][1] == 'NNP') :
            df_res_NNG_NNP = df_res_NNG_NNP.append({'art_id':df_mec.loc[i][0],'token': "".join([df_mec.loc[i][1][j-1][0], df_mec.loc[i][1][j][0] ]) ,'type':'comp_NNG_NNP'},ignore_index=True)
            continue
        df_res_NNG_NNP = df_res_NNG_NNP.append({'art_id':df_mec.loc[i][0], 'token':df_mec.loc[i][1][j][0] ,'type':df_mec.loc[i][1][j][1]}, ignore_index=True)


df_res_NNG_NNP.to_csv('df_res_NNG_NNP_test1.csv', encoding='utf-8-sig', index=False)

print(df_res_NNG_NNP)

     art_id token          type
0         0   erp            SL
1         0  프로그램           NNG
2         0    오류           NNG
3         0  장애조치  comp_NNG_NNP
4         0    방법           NNG
...     ...   ...           ...
6578   1000    고수           NNP
6579   1000     님           XSN
6580   1000     들           XSN
6581   1000     주            VX
6582   1000    세요         EP+EF

[6583 rows x 3 columns]
